# Trabalho 2: Transfer Learning


# **Alunos**

*   André Dorte dos Santos
*   Luiz Massao Miyazaki


## Carregamento dos dados

Os .zips contendo o dataset está no google drive. O código abaixo faz o download, e extrai o zip. Note que você precisará logar no Google para que esse processo seja possível. Caso haja alguma falha ou erro no processo, reinicie o notebook, e tente novamente.

Depois desse processo, teremos no diretório:

*   isic2017-train: 2000 imagens.
*   isic2017-val: 150 imagens.
*   isic2017-test: 650 imagens.



In [ ]:
!pip install PyDrive &> /dev/null

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1EH2tlRi4Iqq8WfLOXl6hMc5jTeiIRzaW"})
downloaded.GetContentFile('isic2017-trainval.zip')
!unzip -q isic2017-trainval.zip

downloaded = drive.CreateFile({'id':"1uP1tCj6-T2FXxk7g7gqoC_SGOhqIQyrM"})
downloaded.GetContentFile('isic2017-test.zip')
!unzip -q isic2017-test.zip

Em cada diretório, temos uma pasta contendo as lesões benignas ('/benign') e outra contendo as lesões malignas ('/malignant'). A separação dessa forma facilita a importação dos dados utilizando o keras.
Estude o uso da ImageDataGenerator do Keras, pois ela facilita o uso de aumentação e pré-processamento dos dados.


# Aumentação de Dados

1. Aplique aumentação de dados nas imagens, escolhendo três transformações. Mostre exemplos das
transformações escolhidas e justifique por que elas são válidas neste dataset (1,0)

In [ ]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from random import sample, seed
seed(42)


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        rotation_range= 20,
        horizontal_flip=True,
        vertical_flip=True)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
train_generator = train_datagen.flow_from_directory(
        'isic2017-train/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'isic2017-val/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)
test_generator = test_datagen.flow_from_directory(
        'isic2017-test/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

Found 2000 images belonging to 2 classes.
Found 150 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


Obtendo tamanhos dos batches

In [ ]:
print(len(train_generator))
print(len(validation_generator))
print(len(test_generator))

63
5
19


## Treinamento dos Modelos

Com os dados carregados corretamente, basta treinar e avaliar o modelo!

In [ ]:
# A métrica oficial da base de dados é a AUC

from sklearn.metrics import roc_auc_score
import numpy as np

# função para avaliar modelos com base na métrica AUC
def evaluation_model(eva_model, eva_data):
  preds = eva_model.predict(eva_data)

  batch_index = 0
  data_list = []
  while batch_index <= eva_data.batch_index:
      _, label = eva_data.next()
      data_list.extend(label)
      batch_index = batch_index + 1

  gt = np.argmax(data_list, axis=1)
  auc = roc_auc_score(gt, preds[:,1])
  print("auc :", auc)

# Modelo Baseline

2. Como baseline, construa e treine um modelo convolucional simples (1,0).


In [ ]:
# Modelo Baseline
from tensorflow.keras import layers

baseline_model = tf.keras.Sequential([
  layers.Conv2D(
    filters=10,
    kernel_size=3
  ),
  layers.GlobalAveragePooling2D(),
  layers.Dense(2, activation='softmax')
])

In [ ]:
# Treinamento Modelo Baseline
sgd = tf.keras.optimizers.SGD(lr=0.001)
baseline_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

baseline_model.fit(
        train_generator,
        steps_per_epoch=63,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10
63/63 [==============================] - 28s 444ms/step - loss: 9.0537 - accuracy: 0.7200 - val_loss: 7.2930 - val_accuracy: 0.8133
Epoch 2/10
63/63 [==============================] - 28s 445ms/step - loss: 4.0320 - accuracy: 0.6935 - val_loss: 2.7611 - val_accuracy: 0.8067
Epoch 3/10
63/63 [==============================] - 28s 440ms/step - loss: 1.6311 - accuracy: 0.7090 - val_loss: 0.8216 - val_accuracy: 0.8133
Epoch 4/10
63/63 [==============================] - 29s 454ms/step - loss: 1.0844 - accuracy: 0.7145 - val_loss: 0.5691 - val_accuracy: 0.8133
Epoch 5/10
63/63 [==============================] - 28s 447ms/step - loss: 0.6853 - accuracy: 0.7615 - val_loss: 0.5148 - val_accuracy: 0.8200
Epoch 6/10
63/63 [==============================] - 28s 445ms/step - loss: 0.5817 - accuracy: 0.7815 - val_loss: 0.6567 - val_accuracy: 0.6400
Epoch 7/10
63/63 [==============================] - 28s 447ms/step - loss: 0.5816 - accuracy: 0.7870 - val_loss: 0.5775 - val_accuracy: 0.7933

In [ ]:
evaluation_model(baseline_model, validation_generator)
score = baseline_model.evaluate(validation_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])

auc : 0.6483333333333332
5/5 [==============================] - 1s 113ms/step - loss: 0.4765 - accuracy: 0.8067
Valid loss: 0.47648289799690247
Valid acc: 0.8066666722297668


3. Escolha uma arquitetura cuja implementação e pesos pré-treinados na ImageNet estejam disponíveis
como extratora de features (1,0)


In [ ]:
# Carregamento do modelo pré-treinado SEM as camadas densas (include_top = False)
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
# model.summary()

4. Inicialmente, descarte a saída original e congele suas as camadas. Adicione uma nova camada de
saída com o número de classes adequadas para este problema (1,0)


In [ ]:
# Congela camadas pré-treinadas
for layer in model.layers:
    layer.trainable = False

# Insere novas camadas no fim da rede para classificação
full_model = tf.keras.Sequential([
  model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation='softmax')
])

# full_model.summary()

In [ ]:
# Treinamento da última camada do modelo congelado
sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.001, nesterov=True)

full_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

full_model.fit(
        train_generator,
        steps_per_epoch=63,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10
63/63 [==============================] - 30s 475ms/step - loss: 0.4998 - accuracy: 0.7925 - val_loss: 0.5336 - val_accuracy: 0.8133
Epoch 2/10
63/63 [==============================] - 29s 462ms/step - loss: 0.4403 - accuracy: 0.8070 - val_loss: 0.4595 - val_accuracy: 0.7933
Epoch 3/10
63/63 [==============================] - 29s 459ms/step - loss: 0.4179 - accuracy: 0.8195 - val_loss: 0.5321 - val_accuracy: 0.8133
Epoch 4/10
63/63 [==============================] - 29s 458ms/step - loss: 0.4097 - accuracy: 0.8200 - val_loss: 0.4860 - val_accuracy: 0.8133
Epoch 5/10
63/63 [==============================] - 28s 451ms/step - loss: 0.3848 - accuracy: 0.8335 - val_loss: 0.4813 - val_accuracy: 0.7467
Epoch 6/10
63/63 [==============================] - 28s 448ms/step - loss: 0.3947 - accuracy: 0.8270 - val_loss: 0.4523 - val_accuracy: 0.8333
Epoch 7/10
63/63 [==============================] - 28s 448ms/step - loss: 0.3801 - accuracy: 0.8385 - val_loss: 0.5251 - val_accuracy: 0.8133

In [ ]:
evaluation_model(full_model, validation_generator)
score = full_model.evaluate(validation_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])

auc : 0.7555555555555555
5/5 [==============================] - 1s 117ms/step - loss: 0.5448 - accuracy: 0.8200
Valid loss: 0.5448468923568726
Valid acc: 0.8199999928474426


# Fine-Tuning
5. Descongele as camadas da rede extratora de features e realize o fine-tuning da rede, continuando o
treinamento (1,5)


In [ ]:
model2 = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

for layer in model2.layers:
    layer.trainable = True

fine_model = tf.keras.Sequential([
  model2,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation='softmax')
])

# Treinamento do modelo descongelado (fine-tuning)
sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.001, nesterov=True)

fine_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

fine_model.fit(
        train_generator,
        steps_per_epoch=63,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=5)

Epoch 1/10
63/63 [==============================] - 32s 508ms/step - loss: 0.4939 - accuracy: 0.7850 - val_loss: 0.5791 - val_accuracy: 0.7867
Epoch 2/10
63/63 [==============================] - 32s 503ms/step - loss: 0.3819 - accuracy: 0.8370 - val_loss: 0.4592 - val_accuracy: 0.8600
Epoch 3/10
63/63 [==============================] - 32s 501ms/step - loss: 0.3490 - accuracy: 0.8470 - val_loss: 0.5786 - val_accuracy: 0.8067
Epoch 4/10
63/63 [==============================] - 32s 504ms/step - loss: 0.3250 - accuracy: 0.8550 - val_loss: 0.6476 - val_accuracy: 0.8267
Epoch 5/10
63/63 [==============================] - 32s 504ms/step - loss: 0.2895 - accuracy: 0.8745 - val_loss: 0.7257 - val_accuracy: 0.8200
Epoch 6/10
63/63 [==============================] - 32s 503ms/step - loss: 0.2863 - accuracy: 0.8780 - val_loss: 0.5742 - val_accuracy: 0.8200
Epoch 7/10
63/63 [==============================] - 32s 501ms/step - loss: 0.2637 - accuracy: 0.8805 - val_loss: 0.7045 - val_accuracy: 0.8133

In [ ]:
evaluation_model(fine_model, validation_generator)
score = fine_model.evaluate(validation_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])

auc : 0.8147222222222222
5/5 [==============================] - 1s 124ms/step - loss: 0.6388 - accuracy: 0.8267
Valid loss: 0.6388445496559143
Valid acc: 0.8266666531562805


# Avaliação do melhor modelo

6. Avalie o melhor modelo no conjunto de teste (0,5)

In [ ]:
print("Modelo Baseline:")
evaluation_model(baseline_model, test_generator)
score = baseline_model.evaluate(test_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])
print()

print("Modelo ResNet50 com camadas congeladas")
evaluation_model(full_model, test_generator)
score = full_model.evaluate(test_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])
print()

print("Modelo ResNet50 com Fine-Tuning")
evaluation_model(fine_model, test_generator)
score = fine_model.evaluate(test_generator)
print('Valid loss:', score[0])
print('Valid acc:', score[1])
print()

Modelo Baseline:
auc : 0.5956008564704217
19/19 [==============================] - 3s 142ms/step - loss: 0.5054 - accuracy: 0.7967
Valid loss: 0.5054080486297607
Valid acc: 0.79666668176651

Modelo ResNet50 com camadas congeladas
auc : 0.7953318822884041
19/19 [==============================] - 3s 154ms/step - loss: 0.4723 - accuracy: 0.8183
Valid loss: 0.4722747206687927
Valid acc: 0.8183333277702332

Modelo ResNet50 com Fine-Tuning
auc : 0.7753180796659058
19/19 [==============================] - 3s 153ms/step - loss: 0.6074 - accuracy: 0.8250
Valid loss: 0.6074337959289551
Valid acc: 0.824999988079071



# Conclusão
7. Compare e interprete os resultados e elabore uma conclusão discutindo os experimentos (4,0).

Nossas análises se encontram no relatório.